In [1]:
data_path = './hw2_data/'

In [2]:
# First lets improve libraries that we are going to be used in this lab session
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import pandas as pd
import io
import sys
import os
import matplotlib.pyplot as plt
plt.switch_backend('agg')
random.seed(134)


In [3]:
MAX_VOCAB_SIZE = 150000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

MAX_SENTENCE_LENGTH = 50
BATCH_SIZE = 128
EMBED_SIZE = 300

In [4]:
# use_cuda = torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

### Dataset Preparation

In [5]:
# y_label_map = {'neutral':0,
#                 'contradiction': 1, 
#                 'entailment': 2};

y_label_map = {'contradiction':0,'neutral':1,'entailment':2}

def get_string_tokenized_data(data_mat_path):
    df = pd.read_csv(os.path.join(data_mat_path), sep="\t")
    data = np.array(df);
    data = data.astype(str)
    
    tokenized_data_x = len(data) * [None];
    y_labels = [y_label_map[x] for x in data[:, 2] ];
    
    all_tokens = [];
    
    for i,x in enumerate(data):
        tokenized_data_x[i] = [x[0].split(), x[1].split()];
        all_tokens += (tokenized_data_x[i][0] + tokenized_data_x[i][1])

    return all_tokens, tokenized_data_x, y_labels
        


# convert token to id in the dataset
def token2index_dataset(tokens_data, token2id):
    indices_data = []
    for tokens1, tokens2 in tokens_data:
        index_list1 = [token2id[token] if token in token2id else UNK_IDX for token in tokens1]
        index_list2 = [token2id[token] if token in token2id else UNK_IDX for token in tokens2]
        indices_data.append([index_list1, index_list2])
    return indices_data


def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(MAX_VOCAB_SIZE))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token


def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    embedding_dict = np.random.randn(MAX_VOCAB_SIZE+2, EMBED_SIZE)
    all_train_tokens = []
    i = 0
    
    for line in fin:
        tokens = line.rstrip().split(' ')
        all_train_tokens.append(tokens[0])
        embedding_dict[i+2] = list(map(float, tokens[1:]))
        i += 1
        if i == MAX_VOCAB_SIZE:
            break
            
    return embedding_dict, all_train_tokens


In [6]:
_, val_data_x, val_data_y = get_string_tokenized_data(os.path.join(data_path, 'mnli_val.tsv'))
_, train_data_x, train_data_y = get_string_tokenized_data(os.path.join(data_path, 'mnli_train.tsv'))

fasttext_embedding_dict, all_fasttext_tokens = load_vectors('wiki-news-300d-1M.vec')

token2id, id2token = build_vocab(all_fasttext_tokens)
train_data_indices = token2index_dataset(train_data_x, token2id)
val_data_indices = token2index_dataset(val_data_x, token2id)


# double checking
print ("Train dataset size is {}".format(len(train_data_indices)))
print ("Val dataset size is {}".format(len(val_data_indices)))

Train dataset size is 20000
Val dataset size is 5000


In [7]:
count = 0;
for x in train_data_indices:
    if 1 in set(x[0]):
        count+=1

In [8]:
count/len(train_data_indices)

0.35685

In [9]:
df = pd.read_csv(os.path.join(data_path, 'mnli_val.tsv'), sep="\t")
data = np.array(df);
data = data.astype(str)
val_genre_list = data[:, 3]

df = pd.read_csv(os.path.join(data_path, 'mnli_train.tsv'), sep="\t")
data = np.array(df);
data = data.astype(str)
train_genre_list = data[:, 3]

In [10]:
unique_genre = list(set(val_genre_list));
print(unique_genre)

['travel', 'government', 'telephone', 'slate', 'fiction']


### Dataset Pytorch Classes

In [11]:
class SNLIDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, data_x, target_list):
        """
        @param data_list: list of newsgroup tokens
        @param target_list: list of newsgroup targets

        """
        self.data_x = data_x;
        self.target_list = target_list
        
        assert(len(data_x) == len(target_list))

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        prem_token_idx = self.data_x[key][0][:MAX_SENTENCE_LENGTH]
        hyp_token_idx = self.data_x[key][1][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [prem_token_idx, hyp_token_idx, label]


def encode_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    prem_data_list = []
    hyp_data_list = []
    label_list = []
    length_list = []
    # print("collate batch: ", batch[0][0])
    # batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
    # padding
    for datum in batch:
        prem_padded_vec = np.pad(np.array(datum[0]),
                                 pad_width=((0, MAX_SENTENCE_LENGTH - len(datum[0]))),
                                 mode="constant", constant_values=0)
        hyp_padded_vec = np.pad(np.array(datum[1]),
                                pad_width=((0, MAX_SENTENCE_LENGTH - len(datum[1]))),
                                mode="constant", constant_values=0)
        prem_data_list.append(prem_padded_vec)
        hyp_data_list.append(hyp_padded_vec)
    return [torch.from_numpy((np.array(prem_data_list))), torch.from_numpy(np.array(hyp_data_list)),
            torch.LongTensor(label_list)]

In [12]:
# train_dataset = SNLIDataset(train_data_indices, train_data_y)
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=BATCH_SIZE,
#                                            collate_fn=encode_collate_func,
#                                            shuffle=True)
# val_dataset = SNLIDataset(val_data_indices, val_data_y)
# val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
#                                            batch_size=BATCH_SIZE,
#                                            collate_fn=encode_collate_func,
#                                            shuffle=True)


In [13]:
assert(len(val_data_indices) == len(val_genre_list))
assert(len(train_data_indices) == len(train_genre_list))

In [14]:
train_dataset_dict = {};
train_loader_dict = {};

for genre in unique_genre:
    req_train_data_indices = np.array(train_data_indices)[ np.where(train_genre_list == genre)[0] ];
    req_train_data_y =  np.array(train_data_y)[ np.where(train_genre_list == genre)[0] ];
    
    train_dataset_dict[genre] = SNLIDataset(req_train_data_indices, req_train_data_y)
    train_loader_dict[genre] = torch.utils.data.DataLoader(dataset=train_dataset_dict[genre],
                                               batch_size=BATCH_SIZE,
                                               collate_fn=encode_collate_func,
                                               shuffle=True)

In [15]:
val_dataset_dict = {};
val_loader_dict = {};

for genre in unique_genre:
    req_val_data_indices = np.array(val_data_indices)[ np.where(val_genre_list == genre)[0] ];
    req_val_data_y =  np.array(val_data_y)[ np.where(val_genre_list == genre)[0] ];
    
    val_dataset_dict[genre] = SNLIDataset(req_val_data_indices, req_val_data_y)
    val_loader_dict[genre] = torch.utils.data.DataLoader(dataset=val_dataset_dict[genre],
                                               batch_size=BATCH_SIZE,
                                               collate_fn=encode_collate_func,
                                               shuffle=True)

## Neural Networks

#### Generic Functions to be used

In [16]:
# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for prem_data, hyp_data, labels in loader:
        prem_data_batch, hyp_data_batch, label_batch = prem_data.to(device), hyp_data.to(device),labels.to(device)
        outputs = F.softmax(model(prem_data_batch, hyp_data_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        total += labels.size(0)
        correct += predicted.eq(label_batch.view_as(predicted)).sum().item()
    return (100 * correct / total)


def plot_acc(train_accs, val_accs, filename):
    f = plt.figure()
    plt.plot(train_accs, label='train');
    plt.plot(val_accs, label='val');
    plt.title(filename);
    plt.legend()

    f.savefig(os.path.join(filename[:3], filename + ".pdf"), bbox_inches='tight')
    plt.show()

In [17]:
def train_and_evaluate(model, regularization, train_loader, val_loader):

#     file_name = '_'.join([model_type, 'kernel_size='+str(kernel_size), 'hidden_size='+str(hidden_size), 'linear_hid_dim='+str(linear_hid_dim), 
#                        'combine_method='+str(combine_method), 'regularization='+str(regularization)]);
#     print('\n'.join([model_type, 'kernel_size='+str(kernel_size), 'hidden_size='+str(hidden_size), 'linear_hid_dim='+str(linear_hid_dim), 
#                        'combine_method='+str(combine_method), 'regularization='+str(regularization)]))
#     sys.stdout.flush()


    
    learning_rate = 1e-3;
    num_epochs = 2;
    
    dropout = (regularization == 'dropout')
    

#     if(model_type == 'cnn'):
#         model = CNN(EMBED_SIZE , hidden_size, MAX_VOCAB_SIZE+2, kernel_size, linear_hid_dim, combine_method, dropout);
#     elif(model_type == 'rnn'):
#         model = RNN(EMBED_SIZE , hidden_size, MAX_VOCAB_SIZE+2,  linear_hid_dim, combine_method, dropout);
#     else:
#         error('invalid model type')
        
    model = model.to(device);
    
    print ("When starting")
    print ("Train Acc {}".format(test_model(train_loader, model)))
    print ("Val Acc {}".format(test_model(val_loader, model)))
    sys.stdout.flush()

    # Criterion and Optimizer
    criterion = torch.nn.CrossEntropyLoss()

    if regularization == 'weight_decay':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val = 0;
    best_state_dict = None;
    
    train_acc_array = [];
    val_acc_array = [];
    
    for epoch in range(num_epochs):

        for i, (prem, hyp, label) in enumerate(train_loader):
            
#             if i>300:
#                 break;
            model.train()
            
            prem_batch, hyp_batch, label_batch = prem.to(device), hyp.to(device), label.to(device);
            
            optimizer.zero_grad()
            
            outputs = model(prem_batch, hyp_batch)
            loss = criterion(outputs, label_batch)
            loss.backward()
            optimizer.step()
            # validate every 300 iterations
            if (i+1) % 300 == 0:
                # validate
                val_acc = test_model(val_loader, model)
                
                if val_acc > best_val:
                    best_state_dict = model.state_dict();
                    best_val = val_acc;
                    
                val_acc_array.append(val_acc);
                train_acc = test_model(train_loader, model);
                train_acc_array.append(train_acc)
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}, Train Acc: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), val_acc, train_acc))
                sys.stdout.flush()
                
#     plot_acc(train_acc_array, val_acc_array, file_name)
    
    print ("After training for {} epochs".format(num_epochs))
    print ("Train Acc {}".format(test_model(train_loader, model)))
    print ("Val Acc {}".format(test_model(val_loader, model)))
    sys.stdout.flush()
    
#     model.load_state_dict(best_state_dict)
    
#     return test_model(train_loader, model), test_model(val_loader, model), model

## CNN Model

In [18]:
class CNN(nn.Module):
    def __init__(self, emb_size, hidden_size, vocab_size, kernel_size, linear_hidden_dim, combine_method, dropout):

        super(CNN, self).__init__()
        
        assert(kernel_size % 2 == 1);
        assert(combine_method in ['concat', 'mul', 'add']);
        
        padding = int( (kernel_size-1)/2 );        

        self.hidden_size = hidden_size;
        self.combine_method = combine_method;
        self.dropout = dropout;
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.embedding.from_pretrained(torch.from_numpy(np.array(fasttext_embedding_dict)).cuda(), freeze = False)
#         self.embedding.weight.data.copy_(torch.from_numpy(np.array(fasttext_embedding_dict).copy()))
    
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size, padding=padding)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size, padding=padding)


        
        if combine_method == 'concat':
            self.linear_layers1 = nn.Linear(hidden_size*2, linear_hidden_dim)
        else:
            self.linear_layers1 = nn.Linear(hidden_size, linear_hidden_dim)
            
        self.linear_layers2 =  nn.Linear(linear_hidden_dim, 3)
        
#         self.xavier_init(self.linear_layers1);
#         self.xavier_init(self.linear_layers2);
        
        if self.dropout:
            self.dropout_layer = nn.Dropout(0.5);
            
        
    def xavier_init(self, layer):
        torch.nn.init.xavier_normal_(layer.weight.data)
        layer.bias.data.fill_(0.01)
    
    def indivual_encoding(self, x):
        batch_size, seq_len = x.size()

        embed = self.embedding(x)
        m = (x == 1)
        m = m.unsqueeze(2).repeat(1, 1, EMBED_SIZE).type(torch.FloatTensor).to(device);
       
        embed = m * embed + (1-m) * embed.clone().detach()
        
        hidden = self.conv1(embed.transpose(1,2)).transpose(1,2)
        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(batch_size, seq_len, hidden.size(-1))

        hidden = self.conv2(hidden.transpose(1,2)).transpose(1,2)
        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(batch_size, seq_len, hidden.size(-1))
        
#         print(hidden.shape)
        hidden = torch.max(hidden, 1)[0]
#         print(hidden.shape)
        
        return hidden
    
    def forward(self, prem, hyp):
        prem_vector = self.indivual_encoding(prem);
        hyp_vector = self.indivual_encoding(hyp);
        
        if self.combine_method == 'concat':
            final_code = torch.cat((prem_vector, hyp_vector), dim=1);
        elif self.combine_method == 'mul':
            final_code = prem_vector * hyp_vector;
        elif self.combine_method == 'add':
            final_code = prem_vector + hyp_vector;
            

        final_code = self.linear_layers1(final_code);
        final_code = F.relu(final_code);
        if self.dropout:
            final_code = self.dropout_layer(final_code);
        final_code = self.linear_layers2(final_code)
            
            
        return final_code

        

## RNN Model

In [19]:
class RNN(nn.Module):
    def __init__(self, emb_size, hidden_size, vocab_size, linear_hidden_dim, combine_method, dropout):

        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.combine_method = combine_method;
        self.dropout = dropout;
        
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.embedding.weight.data.copy_(torch.from_numpy(np.array(fasttext_embedding_dict).copy()))
        
        self.bi_gru = nn.GRU(emb_size, hidden_size, num_layers=1, batch_first=True, bidirectional=True)
        
        if combine_method == 'concat':
            self.linear_layers1 = nn.Linear(hidden_size*2, linear_hidden_dim)
        else:
            self.linear_layers1 = nn.Linear(hidden_size, linear_hidden_dim)
            
        self.linear_layers2 =  nn.Linear(linear_hidden_dim, 3)
        
        if self.dropout:
            self.dropout_layer = nn.Dropout(0.5);
            
    def init_hidden(self, batch_size):
        # Function initializes the activation of recurrent neural net at timestep 0
        # Needs to be in format (num_layers, batch_size, hidden_size)
        hidden = torch.zeros(2, batch_size, self.hidden_size).to(device)
        return hidden
    
    def encode(self, x):
        
        batch_size, seq_len = x.size()
        self.hidden = self.init_hidden(batch_size)
        embed = self.embedding(x)
        m = (x == 1)
        m = m.unsqueeze(2).repeat(1, 1, EMBED_SIZE).type(torch.FloatTensor).to(device)
        embed = m * embed + (1-m) * embed.clone().detach()
        
        # embed = torch.nn.utils.rnn.pack_padded_sequence(embed, lengths.cpu().numpy(), batch_first=True)
        
        output, hidden = self.bi_gru(embed, self.hidden)
        hidden = torch.sum(hidden, dim = 0)
        
#         hidden = hidden.index_select(0, idx_unsort)
        
        return hidden
    
    
    def forward(self, prem, hyp):
        batch_size, seq_len = prem.size()

        prem_vector = self.encode(prem)
        hyp_vector = self.encode(hyp)
        
        if self.combine_method == 'concat':
            final_code = torch.cat((prem_vector, hyp_vector), dim=1);
        elif self.combine_method == 'mul':
            final_code = prem_vector * hyp_vector;
        elif self.combine_method == 'add':
            final_code = prem_vector + hyp_vector;
            

        final_code = self.linear_layers1(final_code);
        final_code = F.relu(final_code);
        if self.dropout:
            final_code = self.dropout_layer(final_code);
        final_code = self.linear_layers2(final_code)
            
            
        return final_code

### Loading Best Models - CNN

In [20]:
best_kernel_size = 3;
best_hidden_size = 100;
best_combine_method = 'concat';
best_regularization_cnn = 'dropout';

In [21]:
best_cnn = torch.load('./cnn_1/best_cnn.pth', map_location={'cuda:0': 'cpu'})

/home/sm7582/.conda/envs/denoising/lib/python3.6/site-packages/torch/serialization.py:391: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


In [22]:
for x in unique_genre:
    val_acc = test_model(val_loader_dict[x], best_cnn)
    print(x, 'Val Acc: ', val_acc)

travel Val Acc:  41.14052953156823
government Val Acc:  41.732283464566926
telephone Val Acc:  43.08457711442786
slate Val Acc:  38.82235528942116
fiction Val Acc:  38.391959798994975


### Loading Best Models - RNN

In [23]:
best_regularization_rnn = 'none';

In [24]:
best_rnn = torch.load('./rnn/best_rnn.pth', map_location={'cuda:0': 'cpu'})

/home/sm7582/.conda/envs/denoising/lib/python3.6/site-packages/torch/serialization.py:391: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


In [25]:
for x in unique_genre:
    val_acc = test_model(val_loader_dict[x], best_rnn)
    print(x, 'Val Acc: ', val_acc)

travel Val Acc:  43.38085539714868
government Val Acc:  46.55511811023622
telephone Val Acc:  47.2636815920398
slate Val Acc:  43.213572854291414
fiction Val Acc:  50.35175879396985


### Fine Turning CNN

In [26]:
for x in unique_genre:
    print('\nGenre: ', x);
    best_cnn = torch.load('./cnn_1/best_cnn.pth', map_location={'cuda:0': 'cpu'})
    train_and_evaluate(best_cnn, best_regularization_cnn,
                                                  train_loader_dict[x], val_loader_dict[x]);


Genre:  travel
When starting
Train Acc 41.27979924717691
Val Acc 41.14052953156823
After training for 2 epochs
Train Acc 60.70263488080301
Val Acc 43.686354378818734

Genre:  government
When starting
Train Acc 40.50991501416431
Val Acc 41.732283464566926
After training for 2 epochs
Train Acc 61.653360803502444
Val Acc 46.35826771653543

Genre:  telephone
When starting
Train Acc 38.99297423887588
Val Acc 43.08457711442786
After training for 2 epochs
Train Acc 60.37470725995316
Val Acc 42.985074626865675

Genre:  slate
When starting
Train Acc 40.188772975658225
Val Acc 38.82235528942116
After training for 2 epochs
Train Acc 56.58221559860904
Val Acc 38.02395209580838

Genre:  fiction
When starting
Train Acc 41.0844629822732
Val Acc 38.391959798994975
After training for 2 epochs
Train Acc 59.019812304483835
Val Acc 39.49748743718593


In [29]:
for x in unique_genre:
    print('\nGenre: ', x);
    best_rnn = torch.load('./rnn/best_rnn.pth', map_location={'cuda:0': 'cpu'})
    train_and_evaluate(best_rnn, best_regularization_rnn,
                                                  train_loader_dict[x], val_loader_dict[x]);


Genre:  travel
When starting
Train Acc 46.148055207026346
Val Acc 43.38085539714868
After training for 2 epochs
Train Acc 61.65621079046424
Val Acc 52.342158859470466

Genre:  government
When starting
Train Acc 48.31315992789081
Val Acc 46.55511811023622
After training for 2 epochs
Train Acc 62.52897244398661
Val Acc 54.82283464566929

Genre:  telephone
When starting
Train Acc 46.51053864168618
Val Acc 47.2636815920398
After training for 2 epochs
Train Acc 60.304449648711945
Val Acc 52.63681592039801

Genre:  slate
When starting
Train Acc 44.63487332339791
Val Acc 43.213572854291414
After training for 2 epochs
Train Acc 58.991554893194234
Val Acc 48.60279441117765

Genre:  fiction
When starting
Train Acc 48.5140771637122
Val Acc 50.35175879396985
After training for 2 epochs
Train Acc 62.877997914494266
Val Acc 53.969849246231156
